# Demo Notebook Exciting Calculation
\
Show basic use cases of the exciting calculation class defined inside the ExcitingWorkflow package.\
Source code: [Gitlab ExcitingWorkflow](https://git.physik.hu-berlin.de/peschelf/excitingworkflow)\
\
Disclaimer: You need the ```excitingtools/bse_xml_generation_and_parsing``` branch from exciting for this. Will get merged soon. Groundstate only allows toplevel attributes, no subelements. Structure has no LDAplusU, dfthalfparam, shell. XS has only the here shown subelements for BSE. Talk to Fabian for more information, if you need more implemented or want to do it yourself.

In [1]:
import os
import shutil
import pathlib

from excitingworkflow.src.exciting_slurm_calculation import ExcitingCalculation
from excitingtools.input.ground_state import ExcitingGroundStateInput
from excitingtools.input.structure import ExcitingStructure
from excitingtools.input.xs import ExcitingXSInput
from excitingtools.runner import BinaryRunner

In [50]:
os.system('rm -r calculation*')  # pay attention here!!
print(os.listdir()) # no calculation directories here

['Demo_Exciting_Calculation.ipynb', 'Demo_Exciting_Slurm_Calculation.ipynb', '.ipynb_checkpoints']


### Exciting Groundstate calculation of LiF
Define the necessary input objects for an exciting calculation

In [3]:
# unit cell lattice of the material
lattice = [[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]]

# Atoms inside the unit cell: every atom own dictionary with species name and position
atoms = [{'species': 'Li', 'position': [0, 0, 0]},
         {'species': 'F', 'position': [0.5, 0.5, 0.5]}]

# Structure object: pass atoms, lattice, species_path and optional kwargs as dictionary
structure = ExcitingStructure(atoms, lattice, './',
                              structure_properties={'autormt': True},
                              crystal_properties={'scale': 7.608})

# Groundstate Object with groundstate attributes
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='fromscratch',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# Exciting runner: pass binary_path, run command (serial or mpi), number of OMP threads, timeout in second
runner1 = BinaryRunner('/home/fabi/code/development/exciting/bin/exciting_smp', './', 4, 500)

# Exciting calculation: pass name, run directory, structure, groundstate, runner
gs_calculation = ExcitingCalculation('test1', 'calculation1', structure,
                                     '/home/fabi/code/exciting/species/', groundstate, runner1)

ExcitingCalculation has three main functions to use:
1. write_inputs()
2. run()
3. parse_output()

In [4]:
gs_calculation.write_inputs()

In [5]:
with open('calculation1/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test1</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0 0 0"> </atom>
		</species>
	</structure>
	<groundstate
	   ngridk="4 4 4"
	   rgkmax="5.0"
	   gmaxvr="10"
	   nempty="5"
	   do="fromscratch"
	   xctype="GGA_PBE_SOL"
	   lmaxmat="12"
	   lmaxvr="12"
	   lmaxapw="12"
	   epschg="0.1"
	   epsengy="0.1"
	   epspot="0.1">
	</groundstate>

</input>




In [6]:
run_result = gs_calculation.run()

In [7]:
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time, 'in s')
print('Success:', run_result.success)

StdOut: b''
StdErr: b''
Return Code: 0
Process time: 15.477900505065918 in s
Success: True


In [8]:
result = gs_calculation.parse_output()
result

{'TOTENERGY.OUT': array([-108.90080119, -107.83916051, -107.32015599, -109.12533784,
        -104.50863651, -107.18950593, -107.03082561, -107.05528806,
        -107.06269123, -107.06309668]),
 'INFO.OUT': {'initialization': {'Lattice vectors (cartesian)': [3.804,
    0.0,
    0.0,
    0.0,
    3.804,
    0.0,
    0.0,
    0.0,
    3.804],
   'Reciprocal lattice vectors (cartesian)': [1.6517311533,
    0.0,
    0.0,
    0.0,
    1.6517311533,
    0.0,
    0.0,
    0.0,
    1.6517311533],
   'Unit cell volume': 55.045462464,
   'Brillouin zone volume': 4.5062790344,
   'Automatic determination of muffin-tin radii': '',
   'global rmt scale': 0.95,
   'Species 1': {'Species symbol': 'F',
    'Species': '1 (F)',
    'parameters loaded from': 'F.xml',
    'name': 'fluorine',
    'nuclear charge': -9.0,
    'electronic charge': 9.0,
    'atomic mass': 34631.97043,
    'muffin-tin radius': 1.7551,
    '# of radial points in muffin-tin': 250,
    'Atomic positions': {'Atom 1': '0.00000000  0.

Note: For the moment, only the `INFO.OUT` and `TOTENERGY.OUT` gets parsed. Extension is really easy (can be done by you or me, just ask me, any feedback is welcome)

In [9]:
result['INFO.OUT']['initialization']['Species 1']['muffin-tin radius']

1.7551

### Exciting BSE calculation of LiF

In [10]:
lattice = [[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]]
atoms = [{'species': 'Li', 'position': [0, 0, 0]},
         {'species': 'F', 'position': [0.5, 0.5, 0.5]}]
structure = ExcitingStructure(atoms, lattice, './',
                              structure_properties={'autormt': True},
                              crystal_properties={'scale': 7.608})
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='fromscratch',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# XS part definition:
# the attributes of the xs element:
xs_attributes = {'broad': 0.327, 'ngridk': [1, 1, 1], 'nempty': 5, 'ngridq': [1, 1, 1], 'gqmax': 2.5,
                 'tappinfo': True, 'tevout': True, 'vkloff': [0.05, 0.03, 0.13]}

# define the necessary sub element trees for BSE calculation
bse_attributes = {'bsetype': 'IP', 'xas': True, 'nstlxas': [1, 5], 'xasatom': 1, 'xasedge': 'K',
                  'xasspecies': '1'}
energywindow_attributes = {'intv': [50, 80], 'points': 100}
screening_attributes = {'screentype': 'full', 'nempty': 5}
# plan_input = ['xsgeneigvec', 'writepmatxs', 'scrgeneigvec', 'scrwritepmat', 'screen', 'scrcoulint', 'exccoulint',
#              'bse']
plan_input = ['xsgeneigvec', 'writepmatxs', 'bse']
qpointset_input = [[0, 0, 0]]

# compose the XS object with xstype and the attributes
xs = ExcitingXSInput("BSE", xs=xs_attributes,
                     BSE=bse_attributes,
                     energywindow=energywindow_attributes,
                     screening=screening_attributes,
                     qpointset=qpointset_input,
                     plan=plan_input)

runner1 = BinaryRunner('/home/fabi/code/development/exciting/bin/exciting_smp', './', 4, 500)

# BSE calculation, pass xs as last parameter
bse_calculation = ExcitingCalculation('test2', 'calculation2', structure,
                                      '/home/fabi/code/exciting/species/', groundstate, runner1, xs)

Modify only one input argument:

In [11]:
bse_calculation.xs.energywindow.points = 200

In [12]:
bse_calculation.write_inputs()
with open('calculation2/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test2</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0 0 0"> </atom>
		</species>
	</structure>
	<groundstate
	   ngridk="4 4 4"
	   rgkmax="5.0"
	   gmaxvr="10"
	   nempty="5"
	   do="fromscratch"
	   xctype="GGA_PBE_SOL"
	   lmaxmat="12"
	   lmaxvr="12"
	   lmaxapw="12"
	   epschg="0.1"
	   epsengy="0.1"
	   epspot="0.1">
	</groundstate>

	<xs
	   xstype="BSE"
	   broad="0.327"
	   ngridk="1 1 1"
	   nempty="5"
	   ngridq="1 1 1"
	   gqmax="2.5"
	   tappinfo="true"
	   tevout="true"
	   vkloff="0.05 0.03 0.13">

		 
		<screening screentype="full" nempty="5"> </screening>
		<BSE
		   bsetype="IP"
		   xas="true"
		   nstlxas="1 5"
		   xasatom="1"
		   xasedge="K"
		

In [13]:
run_result = bse_calculation.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b'\rSolving BSE Eigenvalue Problem:                     100.000\n'
StdErr: b''
Return Code: 0
Process time: 25.01240587234497
Success: True


In [14]:
result_2 = bse_calculation.parse_output()
print(result_2)

{'TOTENERGY.OUT': array([-108.90080119, -107.83916051, -107.32015599, -109.12533784,
       -104.50863651, -107.18950593, -107.03082561, -107.05528806,
       -107.06269123, -107.06309668]), 'INFO.OUT': {'initialization': {'Lattice vectors (cartesian)': [3.804, 0.0, 0.0, 0.0, 3.804, 0.0, 0.0, 0.0, 3.804], 'Reciprocal lattice vectors (cartesian)': [1.6517311533, 0.0, 0.0, 0.0, 1.6517311533, 0.0, 0.0, 0.0, 1.6517311533], 'Unit cell volume': 55.045462464, 'Brillouin zone volume': 4.5062790344, 'Automatic determination of muffin-tin radii': '', 'global rmt scale': 0.95, 'Species 1': {'Species symbol': 'F', 'Species': '1 (F)', 'parameters loaded from': 'F.xml', 'name': 'fluorine', 'nuclear charge': -9.0, 'electronic charge': 9.0, 'atomic mass': 34631.97043, 'muffin-tin radius': 1.7551, '# of radial points in muffin-tin': 250, 'Atomic positions': {'Atom 1': '0.00000000  0.00000000  0.00000000'}}, 'Species 2': {'Species symbol': 'Li', 'Species': '2 (Li)', 'parameters loaded from': 'Li.xml', '

### Exciting BSE calculation on top of groundstate Calculation

In [24]:
xs_attributes = {'broad': 0.327, 'ngridk': [1, 1, 1], 'nempty': 5, 'ngridq': [1, 1, 1], 'gqmax': 2.5,
                     'tappinfo': True, 'tevout': True, 'vkloff': [0.05, 0.03, 0.13]}
bse_attributes = {'bsetype': 'IP', 'xas': True, 'nstlxas': [1, 5], 'xasatom': 1, 'xasedge': 'K',
                  'xasspecies': '1'}
energywindow_attributes = {'intv': [50, 80], 'points': 200}
screening_attributes = {'screentype': 'full', 'nempty': 5}
plan_input = ['xsgeneigvec', 'writepmatxs', 'bse']
qpointset_input = [[0, 0, 0]]
xs = ExcitingXSInput("BSE", xs=xs_attributes,
                     BSE=bse_attributes,
                     energywindow=energywindow_attributes,
                     screening=screening_attributes,
                     qpointset=qpointset_input,
                     plan=plan_input)

runner1 = BinaryRunner('/home/fabi/code/development/exciting/bin/exciting_smp', './', 4, 500)

pure_bse_calculation = ExcitingCalculation('test3', 'calculation3', 'calculation1', 'calculation1',
                                           'calculation1', runner1, xs)

In [25]:
pure_bse_calculation.write_inputs()
with open('calculation3/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test3</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0.0 0.0 0.0"> </atom>
		</species>
	</structure>
	<groundstate
	   ngridk="4 4 4"
	   rgkmax="5.0"
	   gmaxvr="10"
	   nempty="5"
	   do="skip"
	   xctype="GGA_PBE_SOL"
	   lmaxmat="12"
	   lmaxvr="12"
	   lmaxapw="12"
	   epschg="0.1"
	   epsengy="0.1"
	   epspot="0.1">
	</groundstate>

	<xs
	   xstype="BSE"
	   broad="0.327"
	   ngridk="1 1 1"
	   nempty="5"
	   ngridq="1 1 1"
	   gqmax="2.5"
	   tappinfo="true"
	   tevout="true"
	   vkloff="0.05 0.03 0.13">

		 
		<screening screentype="full" nempty="5"> </screening>
		<BSE
		   bsetype="IP"
		   xas="true"
		   nstlxas="1 5"
		   xasatom="1"
		   xasedge="K"
		 

In [26]:
os.listdir('calculation3')

['atoms.xml',
 'EFERMI.OUT',
 'BONDLENGTH_QMT001.OUT',
 'Li.xml',
 'EXCITON',
 'STATE.OUT',
 'info_QMT001.xml',
 'geometry_QMT001.xml',
 'EFERMI_QMT001.OUT',
 'EVECSV_QMT001.OUT',
 'EPSILON',
 'evalcore.xml',
 'Li_scf.xml',
 'GQPOINTS_QMT001.OUT',
 'OCCSV_QMT001.OUT',
 'KPOINTS_QMT001.OUT',
 'EVECFV_QMT001.OUT',
 'INFO_QMT001.OUT',
 'WARNINGS.OUT',
 'APWCMT_QMT001.OUT',
 'LOCMT_QMT001.OUT',
 'eigval.xml',
 'QMTPOINTS.OUT',
 'SIGMA',
 'EVALSV_QMT001.OUT',
 'LOSS',
 'INFOXS.OUT',
 'input.xml',
 'QPOINTS.OUT',
 'F.xml',
 'EIGVAL_QMT001.OUT',
 'PMAT_XS.OUT',
 'F_scf.xml']

In [27]:
run_result = pure_bse_calculation.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b'\rSolving BSE Eigenvalue Problem:                     100.000\n'
StdErr: b''
Return Code: 0
Process time: 10.253006935119629
Success: True


In [28]:
result_3 = pure_bse_calculation.parse_output()
result_3

{'LOSS_BSE-IP_SCR-full_OC22.OUT': {'frequency': [1360.569193,
   1364.650900579,
   1368.732608158,
   1372.814315737,
   1376.896023316,
   1380.977730895,
   1385.059438474,
   1389.141146053,
   1393.222853632,
   1397.304561211,
   1401.38626879,
   1405.467976369,
   1409.549683948,
   1413.631391527,
   1417.713099106,
   1421.794806685,
   1425.876514264,
   1429.958221843,
   1434.039929422,
   1438.121637001,
   1442.20334458,
   1446.285052159,
   1450.366759738,
   1454.448467317,
   1458.530174896,
   1462.611882475,
   1466.693590054,
   1470.775297633,
   1474.857005212,
   1478.938712791,
   1483.02042037,
   1487.102127949,
   1491.183835528,
   1495.265543107,
   1499.347250686,
   1503.428958265,
   1507.510665844,
   1511.592373423,
   1515.674081002,
   1519.755788581,
   1523.83749616,
   1527.919203739,
   1532.000911318,
   1536.082618897,
   1540.164326476,
   1544.246034055,
   1548.327741634,
   1552.409449213,
   1556.491156792,
   1560.572864371,
   1564.654

In [29]:
result_2['EPSILON_BSE-IP_SCR-full_OC11.OUT']['imag_oscillator_strength'] == result_3['EPSILON_BSE-IP_SCR-full_OC11.OUT']['imag_oscillator_strength']

True

### Using the groundstate calculation object
Note: You dont have to specify all objects again, as jupyter stores them

In [30]:
pure_bse_calculation2 = ExcitingCalculation('test4', 'calculation4', gs_calculation, gs_calculation, 
                                            gs_calculation, runner1, xs)

In [31]:
pure_bse_calculation2.write_inputs()
with open('calculation4/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test4</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0 0 0"> </atom>
		</species>
	</structure>
	<groundstate
	   ngridk="4 4 4"
	   rgkmax="5.0"
	   gmaxvr="10"
	   nempty="5"
	   do="skip"
	   xctype="GGA_PBE_SOL"
	   lmaxmat="12"
	   lmaxvr="12"
	   lmaxapw="12"
	   epschg="0.1"
	   epsengy="0.1"
	   epspot="0.1">
	</groundstate>

	<xs
	   xstype="BSE"
	   broad="0.327"
	   ngridk="1 1 1"
	   nempty="5"
	   ngridq="1 1 1"
	   gqmax="2.5"
	   tappinfo="true"
	   tevout="true"
	   vkloff="0.05 0.03 0.13">

		 
		<screening screentype="full" nempty="5"> </screening>
		<BSE
		   bsetype="IP"
		   xas="true"
		   nstlxas="1 5"
		   xasatom="1"
		   xasedge="K"
		   xass

In [32]:
run_result = pure_bse_calculation2.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b'\rSolving BSE Eigenvalue Problem:                     100.000\n'
StdErr: b''
Return Code: 0
Process time: 10.766857147216797
Success: True


In [33]:
result_4 = pure_bse_calculation2.parse_output()
result_4

{'LOSS_BSE-IP_SCR-full_OC22.OUT': {'frequency': [1360.569193,
   1364.650900579,
   1368.732608158,
   1372.814315737,
   1376.896023316,
   1380.977730895,
   1385.059438474,
   1389.141146053,
   1393.222853632,
   1397.304561211,
   1401.38626879,
   1405.467976369,
   1409.549683948,
   1413.631391527,
   1417.713099106,
   1421.794806685,
   1425.876514264,
   1429.958221843,
   1434.039929422,
   1438.121637001,
   1442.20334458,
   1446.285052159,
   1450.366759738,
   1454.448467317,
   1458.530174896,
   1462.611882475,
   1466.693590054,
   1470.775297633,
   1474.857005212,
   1478.938712791,
   1483.02042037,
   1487.102127949,
   1491.183835528,
   1495.265543107,
   1499.347250686,
   1503.428958265,
   1507.510665844,
   1511.592373423,
   1515.674081002,
   1519.755788581,
   1523.83749616,
   1527.919203739,
   1532.000911318,
   1536.082618897,
   1540.164326476,
   1544.246034055,
   1548.327741634,
   1552.409449213,
   1556.491156792,
   1560.572864371,
   1564.654

In [34]:
result_4['EPSILON_BSE-IP_SCR-full_OC11.OUT']['imag_oscillator_strength'] == result_3['EPSILON_BSE-IP_SCR-full_OC11.OUT']['imag_oscillator_strength']

True

### Error:

In [35]:
# Groundstate Object with groundstate attributes
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='keks',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# Exciting calculation: pass name, run directory, structure, groundstate, runner
error_calculation = ExcitingCalculation('test5', 'calculation5', structure,
                                   '/home/fabi/code/exciting/species/', groundstate, runner1)

In [36]:
error_calculation.write_inputs()
run_result = error_calculation.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b" Parser ERROR: 'keks                                                                            ' is not valid selection for do \n"
StdErr: b''
Return Code: 0
Process time: 0.004721641540527344
Success: True


Error handling in exciting not existing... yet!

### Parse input.xml and modify just slightly:

In [40]:
from excitingtools.parser.input_parser import parse_input_xml_to_objects

In [41]:
input_dict = parse_input_xml_to_objects('calculation1/input.xml')
input_dict

{'structure': <excitingtools.input.structure.ExcitingStructure at 0x7f2d9ddcd8d0>,
 'ground_state': <excitingtools.input.ground_state.ExcitingGroundStateInput at 0x7f2d9ddcd570>}

In [43]:
structure = input_dict['structure']
ground_state = input_dict['ground_state']
ground_state.__dict__

{'name': 'groundstate',
 'ngridk': '4 4 4',
 'rgkmax': '5.0',
 'gmaxvr': '10',
 'nempty': '5',
 'do': 'fromscratch',
 'xctype': 'GGA_PBE_SOL',
 'lmaxmat': '12',
 'lmaxvr': '12',
 'lmaxapw': '12',
 'epschg': '0.1',
 'epsengy': '0.1',
 'epspot': '0.1'}

In [45]:
ground_state.ngridk = [5, 5, 5]

In [46]:
new_calculation = ExcitingCalculation('test6', 'calculation6', structure,
                                   '/home/fabi/code/exciting/species/', ground_state, runner1)

In [47]:
new_calculation.write_inputs()
with open('calculation6/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test6</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0.0 0.0 0.0"> </atom>
		</species>
	</structure>
	<groundstate
	   ngridk="5 5 5"
	   rgkmax="5.0"
	   gmaxvr="10"
	   nempty="5"
	   do="fromscratch"
	   xctype="GGA_PBE_SOL"
	   lmaxmat="12"
	   lmaxvr="12"
	   lmaxapw="12"
	   epschg="0.1"
	   epsengy="0.1"
	   epspot="0.1">
	</groundstate>

</input>




In [48]:
run_result = new_calculation.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b''
StdErr: b''
Return Code: 0
Process time: 15.345229387283325
Success: True


In [49]:
result_5 = new_calculation.parse_output()
result_5

{'TOTENERGY.OUT': array([-108.88209273, -107.82497797, -107.31041902, -109.13483057,
        -104.47976759, -107.18534469, -107.02993256, -107.05405742,
        -107.06308437, -107.06297842]),
 'INFO.OUT': {'initialization': {'Lattice vectors (cartesian)': [3.804,
    0.0,
    0.0,
    0.0,
    3.804,
    0.0,
    0.0,
    0.0,
    3.804],
   'Reciprocal lattice vectors (cartesian)': [1.6517311533,
    0.0,
    0.0,
    0.0,
    1.6517311533,
    0.0,
    0.0,
    0.0,
    1.6517311533],
   'Unit cell volume': 55.045462464,
   'Brillouin zone volume': 4.5062790344,
   'Automatic determination of muffin-tin radii': '',
   'global rmt scale': 0.95,
   'Species 1': {'Species symbol': 'F',
    'Species': '1 (F)',
    'parameters loaded from': 'F.xml',
    'name': 'fluorine',
    'nuclear charge': -9.0,
    'electronic charge': 9.0,
    'atomic mass': 34631.97043,
    'muffin-tin radius': 1.7551,
    '# of radial points in muffin-tin': 250,
    'Atomic positions': {'Atom 1': '0.00000000  0.